In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [55]:
class VeggieVision:
    def __init__(self, base_model, input_shape=(224, 224, 3)):
        # Set parameters
        self.input_shape = input_shape
        self.base_model = base_model
        self.model = None
        self.datagen = None

        # Build the model
        self.build_model()

    def build_model(self):
        
        
        # Freeze the base model layers initially
        self.base_model.trainable = False

       # 2. Build the Model
        self.model = models.Sequential()

        # Add the base model (MobileNetV2)
        self.model.add(self.base_model)

        self.model.add(layers.Flatten())

        # Add an additional Dense layer with 128 neurons and ReLU activation
        self.model.add(layers.Dense(1024, activation='relu'))


        self.model.add(layers.Dense(256, activation = "relu"))


        #   Final Dense layer with a single neuron for regression (predicting weight)
        self.model.add(layers.Dense(1, activation='linear'))
        

    def augment_data(self, X_train):
        # 5. Data Augmentation
        # Create an ImageDataGenerator for real-time data augmentation
        self.datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            rotation_range=180,         # Rotate images by up to 90 degrees
            width_shift_range=0.1,     # Shift image horizontally by up to 20% of the width
            height_shift_range=0.1,    # Shift image vertically by up to 20% of the height
            shear_range=0.1,           # Shear the image by up to 10%
            zoom_range=0.1,            # Zoom into the image by up to 20%
            horizontal_flip=True,      # Randomly flip images horizontally
            fill_mode='nearest'        # Fill in missing pixels with the nearest filled value
        )

        # Fit the ImageDataGenerator on the training data
        self.datagen.fit(X_train)

    def fit(self, X, y, batch_size=32, epochs=20, learning_rate=1e-4, validation_split=0.2, fine_tune=False):
        # 4. Split Data into Training and Validation Sets
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=validation_split, random_state=42)

        # Augment the data
        self.augment_data(X_train)

        # Compile the model with the provided learning rate
        self.model.compile(optimizer=Adam(learning_rate=learning_rate), 
                           loss='mean_squared_error', 
                           metrics=['mae'])

        # 6. Train the Model
        # Use the data generator to augment and train the model
        history = self.model.fit(self.datagen.flow(X_train, y_train, batch_size=batch_size),
                                 validation_data=(X_val, y_val), 
                                 epochs=epochs)

        # Evaluate the model on the validation set
        test_loss, test_mae = self.model.evaluate(X_val, y_val)
        print(f'Test MAE after initial training: {test_mae}')

        # 8. Fine-tuning the model (if requested)
        if fine_tune:
            self.fine_tune(X_train, y_train, X_val, y_val, batch_size, epochs, learning_rate)

        return history

    def fine_tune(self, X_train, y_train, X_val, y_val, batch_size=32, epochs=10, fine_tune_learning_rate=1e-5):
        # Unfreeze the base MobileNetV2 model's layers
        self.base_model.trainable = True

        # Compile with a lower learning rate for fine-tuning
        self.model.compile(optimizer=Adam(learning_rate=fine_tune_learning_rate), 
                           loss='mean_squared_error', 
                           metrics=['mae'])

        # Fine-tune the model with augmented data
        fine_tune_history = self.model.fit(self.datagen.flow(X_train, y_train, batch_size=batch_size),
                                           validation_data=(X_val, y_val), 
                                           epochs=epochs)

        # Evaluate after fine-tuning
        test_loss, test_mae = self.model.evaluate(X_val, y_val)
        print(f'Test MAE after fine-tuning: {test_mae}')

        return fine_tune_history
    
    def evaluate(self, X, y):
        test_loss, test_mae = self.model.evaluate(X, y)
        print(f'MAE: {test_mae}')
        return test_loss, test_mae


In [4]:
X = np.load("../data/processed_data/X.npy")
y = np.load("../data/processed_data/y.npy")

In [56]:
base_model = MobileNetV2(input_shape=(224,224, 3), include_top=False, weights='imagenet')
veggie_mnv2_bb = VeggieVision(base_model)

In [69]:
veggie_mnv2_bb.fit(X,y, learning_rate = 1e-6, fine_tune = True)

Epoch 1/20
19/19 [==============================] - 25s 1s/step - loss: 8016.3203 - mae: 23.1250 - val_loss: 811.2410 - val_mae: 19.8060
Epoch 2/20


KeyboardInterrupt: 

In [68]:
damnidt = veggie_mnv2_bb
damnitd.model.save('dadmnit_model.h5')


In [64]:
dam = veggie_mnv2_bb
damn.model.save('dadmn_model.h5')


In [71]:
damnit.evaluate(X,y)

23/23 [==============================] - 13s 554ms/step - loss: 4394.8379 - mae: 21.9176
MAE: 21.917591094970703


(4394.837890625, 21.917591094970703)

In [34]:
base_model = MobileNetV2(input_shape=(224,224, 3), include_top=False, weights='imagenet')
veggie_mnv2_bigbig = VeggieVision(base_model)

In [38]:
veggie_mnv2_bigbig.fit(X,y, learning_rate = 1e-6, fine_tune = True)

Epoch 1/20
19/19 [==============================] - 20s 840ms/step - loss: 9021.1709 - mae: 27.0486 - val_loss: 977.8801 - val_mae: 21.5109
Epoch 2/20
19/19 [==============================] - 15s 761ms/step - loss: 9190.2900 - mae: 27.0290 - val_loss: 977.4487 - val_mae: 21.5070
Epoch 3/20
19/19 [==============================] - 14s 754ms/step - loss: 8766.9678 - mae: 26.5183 - val_loss: 977.4348 - val_mae: 21.5010
Epoch 4/20
19/19 [==============================] - 14s 750ms/step - loss: 8850.1240 - mae: 27.4496 - val_loss: 977.1920 - val_mae: 21.5031
Epoch 5/20
19/19 [==============================] - 15s 760ms/step - loss: 8798.7246 - mae: 26.4522 - val_loss: 977.1414 - val_mae: 21.4989
Epoch 6/20
19/19 [==============================] - 15s 776ms/step - loss: 8712.3955 - mae: 26.8626 - val_loss: 976.8643 - val_mae: 21.5000
Epoch 7/20
19/19 [==============================] - 15s 770ms/step - loss: 9194.8916 - mae: 27.1538 - val_loss: 976.6826 - val_mae: 21.4991
Epoch 8/20
19/19 [==

KeyboardInterrupt: 

In [39]:
veggie_mnv2_bigbig.evaluate(X, y)

23/23 [==============================] - 12s 515ms/step - loss: 7093.5547 - mae: 25.8197
MAE: 25.819664001464844


(7093.5546875, 25.819664001464844)

In [28]:
base_model = MobileNetV2(input_shape=(224,224, 3), include_top=False, weights='imagenet')
veggie_mnv2_big = VeggieVision(base_model)

In [32]:
veggie_mnv2_big.fit(X,y, learning_rate = 1e-5, fine_tune= True)

Epoch 1/20
19/19 [==============================] - 19s 806ms/step - loss: 9509.0088 - mae: 25.0226 - val_loss: 1046.1040 - val_mae: 22.3595
Epoch 2/20
19/19 [==============================] - 14s 748ms/step - loss: 10081.6904 - mae: 25.5655 - val_loss: 1039.8589 - val_mae: 22.2927
Epoch 3/20
19/19 [==============================] - 14s 750ms/step - loss: 10133.2607 - mae: 25.2103 - val_loss: 1021.2952 - val_mae: 22.0675
Epoch 4/20
19/19 [==============================] - 14s 736ms/step - loss: 9874.7236 - mae: 26.5130 - val_loss: 1020.6499 - val_mae: 22.0585
Epoch 5/20
 9/19 [=============>................] - ETA: 6s - loss: 20185.7500 - mae: 28.7149

KeyboardInterrupt: 

In [23]:
base_model = MobileNetV2(input_shape=(224,224, 3), include_top=False, weights='imagenet')
veggie_mnv2_bm = VeggieVision(base_model)

In [26]:
veggie_mnv2_bm.fit(X,y, learning_rate = 1e-6, fine_tune = True)

Epoch 1/20
19/19 [==============================] - 19s 872ms/step - loss: 11224.3750 - mae: 27.5177 - val_loss: 1065.8956 - val_mae: 22.9926
Epoch 2/20
19/19 [==============================] - 15s 766ms/step - loss: 11339.2041 - mae: 29.2789 - val_loss: 1066.2828 - val_mae: 22.9936
Epoch 3/20
19/19 [==============================] - 15s 762ms/step - loss: 10529.9727 - mae: 27.9604 - val_loss: 1065.9048 - val_mae: 22.9932
Epoch 4/20
19/19 [==============================] - 14s 752ms/step - loss: 11203.5811 - mae: 27.8386 - val_loss: 1066.5621 - val_mae: 22.9980
Epoch 5/20
19/19 [==============================] - 14s 755ms/step - loss: 11414.5312 - mae: 28.2445 - val_loss: 1065.9891 - val_mae: 22.9964
Epoch 6/20
19/19 [==============================] - 15s 764ms/step - loss: 11250.6738 - mae: 28.9828 - val_loss: 1065.7697 - val_mae: 22.9935
Epoch 7/20
13/19 [===================>..........] - ETA: 3s - loss: 1180.2438 - mae: 25.1589

KeyboardInterrupt: 

In [18]:
base_model = MobileNetV2(input_shape=(224,224, 3), include_top=False, weights='imagenet')
veggie_mnv2_mm = VeggieVision(base_model)

In [21]:
veggie_mnv2_mm.fit(X,y, learning_rate = 1e-6, fine_tune= True)

Epoch 1/20
19/19 [==============================] - 19s 786ms/step - loss: 10785.6582 - mae: 27.3547 - val_loss: 1066.7749 - val_mae: 22.5157
Epoch 2/20
19/19 [==============================] - 14s 727ms/step - loss: 10997.1260 - mae: 28.3155 - val_loss: 1064.7516 - val_mae: 22.4998
Epoch 3/20
 2/19 [==>...........................] - ETA: 11s - loss: 1271.5073 - mae: 25.4798

KeyboardInterrupt: 

In [12]:
base_model = MobileNetV2(input_shape=(224,224, 3), include_top=False, weights='imagenet')
veggie_mnv2_m = VeggieVision(base_model)

In [15]:
veggie_mnv2_m.fit(X,y, learning_rate = 1e-5, fine_tune=True)

Epoch 1/20
19/19 [==============================] - 18s 768ms/step - loss: 11257.1055 - mae: 27.0282 - val_loss: 1024.8042 - val_mae: 22.1567
Epoch 2/20
19/19 [==============================] - 14s 717ms/step - loss: 11228.5625 - mae: 27.2025 - val_loss: 1028.2767 - val_mae: 22.1896
Epoch 3/20
19/19 [==============================] - 14s 721ms/step - loss: 11085.3389 - mae: 27.6297 - val_loss: 1047.3746 - val_mae: 22.3205
Epoch 4/20
19/19 [==============================] - 14s 730ms/step - loss: 11450.3984 - mae: 26.5998 - val_loss: 1045.3978 - val_mae: 22.3209
Epoch 5/20
19/19 [==============================] - 14s 736ms/step - loss: 11389.8086 - mae: 27.5232 - val_loss: 1042.2882 - val_mae: 22.2858
Epoch 6/20
 4/19 [=====>........................] - ETA: 9s - loss: 828.7551 - mae: 21.1698 

KeyboardInterrupt: 

In [5]:
base_model = MobileNetV2(input_shape=(224,224, 3), include_top=False, weights='imagenet')
veggie_mnv2_s = VeggieVision(base_model)

In [10]:
veggie_mnv2_s.fit(X,y, learning_rate = 1e-5, fine_tune = True)

Epoch 1/20
19/19 [==============================] - 18s 772ms/step - loss: 10783.2324 - mae: 27.6232 - val_loss: 1123.8319 - val_mae: 22.7881
Epoch 2/20
19/19 [==============================] - 15s 776ms/step - loss: 10488.9873 - mae: 25.8228 - val_loss: 1109.8132 - val_mae: 22.6903
Epoch 3/20
19/19 [==============================] - 15s 761ms/step - loss: 10519.0117 - mae: 26.1946 - val_loss: 1103.7384 - val_mae: 22.6412
Epoch 4/20
19/19 [==============================] - ETA: 0s - loss: 10999.8486 - mae: 27.2179

KeyboardInterrupt: 

In [61]:
base_model = MobileNetV2(input_shape=(224,224, 3), include_top=False, weights='imagenet')
veggie_mnv2_extra = VeggieVision(base_model)

In [64]:
veggie_mnv2_extra.fit(X,y, learning_rate = 1e-5, fine_tune=True)

Epoch 1/20
19/19 [==============================] - 17s 767ms/step - loss: 11126.0820 - mae: 28.1240 - val_loss: 1013.8907 - val_mae: 21.7296
Epoch 2/20
19/19 [==============================] - 14s 712ms/step - loss: 9534.6074 - mae: 25.1830 - val_loss: 1010.8397 - val_mae: 21.7444
Epoch 3/20
19/19 [==============================] - 13s 698ms/step - loss: 10584.8652 - mae: 26.5595 - val_loss: 1017.6968 - val_mae: 21.7540
Epoch 4/20
19/19 [==============================] - 13s 700ms/step - loss: 9946.2090 - mae: 25.5918 - val_loss: 1008.6127 - val_mae: 21.7069
Epoch 5/20
19/19 [==============================] - 13s 703ms/step - loss: 9055.1914 - mae: 27.0399 - val_loss: 1008.7377 - val_mae: 21.7097
Epoch 6/20
19/19 [==============================] - 15s 769ms/step - loss: 10585.0869 - mae: 27.1011 - val_loss: 1004.8294 - val_mae: 21.6923
Epoch 7/20
 8/19 [===========>..................] - ETA: 6s - loss: 19513.6328 - mae: 31.7834

KeyboardInterrupt: 

In [54]:
base_model = MobileNetV2(input_shape=(224,224, 3), include_top=False, weights='imagenet')
veggie_mnv2_pro = VeggieVision(base_model)

In [58]:
veggie_mnv2_pro.fit(X,y, learning_rate = 1e-5, fine_tune=True)

Epoch 1/20
19/19 [==============================] - 17s 756ms/step - loss: 8472.1367 - mae: 24.8424 - val_loss: 1090.3677 - val_mae: 21.9171
Epoch 2/20
19/19 [==============================] - 13s 695ms/step - loss: 8747.0801 - mae: 26.6176 - val_loss: 1108.3489 - val_mae: 22.1646
Epoch 3/20
19/19 [==============================] - 14s 723ms/step - loss: 8650.6650 - mae: 24.5839 - val_loss: 1104.5146 - val_mae: 22.1029
Epoch 4/20
19/19 [==============================] - 14s 736ms/step - loss: 9292.9736 - mae: 25.8860 - val_loss: 1106.6066 - val_mae: 22.1254
Epoch 5/20
19/19 [==============================] - 14s 721ms/step - loss: 8770.3369 - mae: 26.2224 - val_loss: 1100.1656 - val_mae: 22.0705
Epoch 6/20
 8/19 [===========>..................] - ETA: 6s - loss: 21378.2520 - mae: 30.1261

KeyboardInterrupt: 

In [48]:
base_model = MobileNetV2(input_shape=(224,224, 3), include_top=False, weights='imagenet')
veggie_mnv2_plus = VeggieVision(base_model)

In [52]:
veggie_mnv2_plus.fit(X,y, learning_rate = 1e-5, fine_tune=True)

Epoch 1/20
19/19 [==============================] - 17s 749ms/step - loss: 8710.0449 - mae: 24.6279 - val_loss: 1067.0911 - val_mae: 22.4756
Epoch 2/20
19/19 [==============================] - 13s 690ms/step - loss: 8703.8311 - mae: 26.4929 - val_loss: 1061.1757 - val_mae: 22.4579
Epoch 3/20
19/19 [==============================] - 14s 715ms/step - loss: 8253.8916 - mae: 25.2616 - val_loss: 1054.8967 - val_mae: 22.4445
Epoch 4/20
19/19 [==============================] - 13s 702ms/step - loss: 8553.9551 - mae: 25.3683 - val_loss: 1054.9999 - val_mae: 22.4753
Epoch 5/20
19/19 [==============================] - 14s 740ms/step - loss: 8309.3369 - mae: 25.8725 - val_loss: 1056.1917 - val_mae: 22.4764
Epoch 6/20
19/19 [==============================] - 14s 710ms/step - loss: 8665.3555 - mae: 26.9874 - val_loss: 1058.3910 - val_mae: 22.4871
Epoch 7/20
19/19 [==============================] - 13s 699ms/step - loss: 8568.2314 - mae: 25.7951 - val_loss: 1064.0902 - val_mae: 22.5067
Epoch 8/20
14

KeyboardInterrupt: 

In [41]:
base_model = MobileNetV2(input_shape=(224,224, 3), include_top=False, weights='imagenet')
veggie_mnv2_full = VeggieVision(base_model)

In [ ]:
veggie_mnv2_full.fit(X,y, learning_rate = 1e-6, fine_tune=True)

In [46]:
veggie_mnv2_full.evaluate(X, y)

23/23 [==============================] - 12s 516ms/step - loss: 6440.0679 - mae: 24.0880
MAE: 24.087970733642578


(6440.06787109375, 24.087970733642578)

In [20]:
base_model = MobileNetV2(input_shape=(224,224, 3), include_top=False, weights='imagenet')
veggie_mnv2 = VeggieVision(base_model)

In [23]:
veggie_mnv2.fit(X,y, learning_rate = 1e-5, fine_tune=True)

Epoch 1/20
19/19 [==============================] - 17s 750ms/step - loss: 11651.5918 - mae: 29.3566 - val_loss: 1159.2534 - val_mae: 24.0608
Epoch 2/20
19/19 [==============================] - 14s 726ms/step - loss: 11802.8789 - mae: 30.0735 - val_loss: 1158.1953 - val_mae: 24.0130
Epoch 3/20
19/19 [==============================] - 13s 693ms/step - loss: 11934.6904 - mae: 29.6028 - val_loss: 1153.2426 - val_mae: 23.9724
Epoch 4/20
19/19 [==============================] - 14s 737ms/step - loss: 11734.7930 - mae: 29.8193 - val_loss: 1146.6461 - val_mae: 23.9537
Epoch 5/20
19/19 [==============================] - 14s 730ms/step - loss: 11477.7510 - mae: 29.3631 - val_loss: 1143.1345 - val_mae: 23.9151
Epoch 6/20
19/19 [==============================] - 14s 715ms/step - loss: 11853.0762 - mae: 29.7795 - val_loss: 1140.0607 - val_mae: 23.8996
Epoch 7/20
19/19 [==============================] - 14s 724ms/step - loss: 12002.5322 - mae: 30.0662 - val_loss: 1141.7993 - val_mae: 23.8844
Epoch 

KeyboardInterrupt: 